# Automatic data downloading

* In this example we will show how one can download data from a jupyter notebook.
* The [PhysOcean](https://github.com/gher-ulg/PhysOcean.jl) module provides ways to automatically download data from the World Ocean Database and from CMEMS In-Situ TAC.
* This module can be installed by: 

```julia
Pkg.add("PhysOcean")
```

Import the necessary packages

In [3]:
using PyPlot              # Visualization package
using PhysOcean           # Download data from the World Ocean Database
using divand              # DIVAnd 

Define the time and geospatial bounding box for downloading the data

In [4]:
# resolution (the resolution is only used for DIVAnd analyses)
dx = dy = 0.25   # medium size test 

# vectors defining the longitude and latitudes grids
# Here longitude and latitude correspond to the Mediterranean Sea
lonr = -7:dx:37
latr = 30:dy:46

# time range of the in-situ data
timerange = [Date(2016,1,1),Date(2016,12,31)];

In [5]:
# Name of the variable
varname = "Temperature"

"Temperature"

Please use your own email address (!) 😉     
It is only use to get notified once the dataset is ready.

In [6]:
# Email for downloading the data
email = "ctroupin@ulg.ac.be"

"ctroupin@ulg.ac.be"

Define the directory where the results will be saved.    
The tilde ~ will be replaced (expanded) by your home user directory.      
Create this path if necessary (including parent path).

In [7]:
basedir = expanduser("~/Downloads/WOD/Med-2016-2")
mkpath(basedir)

## Download the data

* World Ocean Database: example for bulk access data by simulating a web-user.
* Downloading can take several tens of minutes.
* SeaDataNet will provide a dedicated machine-to-machine interface during the SeaDataCloud project

In [8]:
?WorldOceanDatabase.download

LoadError: [91mUndefVarError: WorldOceanDatabase not defined[39m

* comment this line (or skip this cell) if the data has already been downloaded

In [ ]:
WorldOceanDatabase.download(lonr,latr,timerange,varname,email,basedir)

# Load data
* Load the data into memory and perform (potentially) an additional subsetting

In [ ]:
# load all data under basedir as a double-precision floating point variable
val,lon,lat,depth,time,ids = WorldOceanDatabase.load(Float64,basedir,varname)

Additional sub-setting based on time and depth for plotting

In [ ]:
# depth range levels
depthr = [0.,20.]
# month range (January to March)

timer = [1,3]
# additional sub-setting and discard bogus negative temperatures
sel = ((val .> 0 )
       .& (minimum(depthr) .<= depth .<= maximum(depthr))
       .& (minimum(timer) .<= Dates.month.(time) .<= maximum(timer)))

valsel = val[sel]
lonsel = lon[sel]
latsel = lat[sel]
depthsel = depth[sel]
timesel = time[sel]
idssel = ids[sel];

In [ ]:
checkobs((lonsel,latsel,depthsel,timesel),valsel,idssel)

Number of data points

In [9]:
length(valsel)

LoadError: [91mUndefVarError: valsel not defined[39m

Download bathymetry for plotting

In [ ]:
bathname = "gebco_30sec_16.nc"

if !isfile(bathname)
    download("https://b2drop.eudat.eu/s/o0vinoQutAC7eb0/download",bathname)
else
    info("Bathymetry file already downloaded")
end

bathisglobal = true

# Extract the bathymetry for plotting
bx,by,b = divand.extract_bath(bathname,bathisglobal,lonr,latr);

In [ ]:
pcolor(bx,by,b')
#contourf(bx,by,b', levels = [-1e5,0],colors = [[.5,.5,.5]])
# compute and set the correct aspect ratio
aspect_ratio = 1/cos(mean(latr) * pi/180)
gca()[:set_aspect](aspect_ratio)
colorbar(orientation = "horizontal")

Plotting gotchas
* `colorbar()` referes to the last added item. If the last added item is the land-sea mask, the colorbar will be all gray
* maps look nicer if you set the corresponding aspect ratio based on the average latitude `mean(latr)`.

In [ ]:
contourf(bx,by,b', levels = [-1e5,0],colors = [[.5,.5,.5]])
scatter(lonsel,latsel,10,valsel); 

# compute and set the correct aspect ratio
aspect_ratio = 1/cos(mean(latr) * pi/180)
gca()[:set_aspect](aspect_ratio)
colorbar(orientation = "horizontal")